In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random


parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="webqsp", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [142]:
data = datas[11]
question = data[question_string]
topics = data['topic_entity']
for topic in topics:
    topic_name = topics[topic]

In [174]:

head_relations = execurte_sparql(sparql_head_relations % topic)
head_relations = filter_relations(head_relations)

prompt = relations_reduced_prompt.format(5, question, topic_name, head_relations)
response = run_llm(prompt, 0.1, args.max_length, args.openai_api_key, args.llm)
        
response_list = re.sub(r'\n[0-99]?(spoilers)?', ' ', response.lower()).split(' ')
response_list = [i.strip("'[]") for i in response_list if i.count('.') > 1]
reduced_relations = []
for relation in head_relations:
    if relation in response_list:
        reduced_relations.append(relation)






In [175]:
print(response)

  Sure, here are the top 5 relations relevant to the topic "Drew Brees" based on the question "where did drew brees go to college wikianswers?":

1. 'people.person.education' - This relation represents the educational background of a person, which is relevant to the question about Drew Brees' college education.
2. 'american_football.football_player.position_s' - This relation represents the position played by a football player, which is relevant to Drew Brees' career as a football quarterback.
3. 'american_football.football_player.games' - This relation represents the number of games played by a football player, which is relevant to Drew Brees' longevity and experience in the NFL.
4. 'american_football.football_player.passing' - This relation represents the passing statistics of a football player, which is relevant to Drew Brees' role as a quarterback.
5. 'people.person.date_of_birth' - This relation represents the date of birth of a person, which is relevant to Drew Brees' age and per

In [176]:
response_list

['people.person.education',
 'american_football.football_player.position_s',
 'american_football.football_player.games',
 'american_football.football_player.passing',
 'people.person.date_of_birth',
 'people.person.education',
 'american_football.football_player.position_s',
 'american_football.football_player.games',
 'american_football.football_player.passing',
 'people.person.date_of_birth']

In [154]:
print(prompt)

Based on the question, please select the most relevant relations of the topic and just return at most 5 relations as a list.

question :where did drew brees go to college wikianswers?
topic: Drew Brees
relations: ['american_football.football_player.forty_yard_dash_time', 'award.award_nominee.award_nominations', 'award.award_winner.awards_won', 'kg.object_profile.prominent_type', 'people.person.children', 'people.person.date_of_birth', 'people.person.education', 'people.person.gender', 'people.person.height_meters', 'people.person.nationality', 'people.person.parents', 'people.person.place_of_birth', 'people.person.places_lived', 'people.person.profession', 'people.person.spouse_s', 'people.person.weight_kg', 'sports.pro_athlete.sports_played_professionally', 'sports.pro_athlete.teams', 'sports.pro_athlete.career_start', 'american_football.football_player.position_s', 'base.schemastaging.athlete_extra.salary', 'symbols.namesake.named_after', 'tv.tv_actor.guest_roles', 'base.schemastagin

In [137]:
entities_id, entities_name = get_entities(topic, relations)

In [138]:
direct_propagate_prompt = """ Given the question and {} related facts of its topic, please summarize each fact while keeping anything useful to answer the question and return each summarized fact as a list.

question: {}

fact:"""
def direct_propagate(question, topic, relations, entities_name, args):
    n = len(relations)
    line_breaks = ['\n{}. '.format(i+1) for i in range(n)]
    prompt = direct_propagate_prompt.format(n, question)
    for i in range(len(relations)):
        prompt += '{}the topic {} has relation {} with following entities {}. '.format(line_breaks[i], topic, relations[i], entities_name[i])
    response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm)
    while re.findall(r'\n[0-99]. ', response) != line_breaks:
        print('Propagation format unmatched. Retrying...')
        print(response)
        response = run_llm(prompt, 0.5, args.max_length, args.openai_api_key, args.llm)
    
    output = [i[i.find(" ")+1:] for i in response.split('\n') if re.match("^[0-99]. ", i)] # select string start with number. and remove it e.g. 1. 


    return prompt, output

prompt, topic_prop = direct_propagate(question, topic_name, relations, entities_name, args)

In [139]:
print(prompt)

 Given the question and 5 related facts of its topic, please summarize each fact while keeping anything useful to answer the question and return each summarized fact as a list.

question: who influenced samuel taylor coleridge?

fact:
1. the topic Samuel Taylor Coleridge has relation book.author.works_written with following entities ['Christabel', 'complete poetical works of Samuel Taylor Coleridge', 'Kubla Khan', 'poems and dramatic works of Samuel Taylor Coleridge', 'Table talk', 'The Notebooks of Samuel Taylor Coleridge, Volume 3', 'The Notebooks of Samuel Taylor Coleridge, Volume 4 : 1819-1826', 'The Notebooks of Samuel Taylor Coleridge, Volume 5: 1827-1834', 'Lyrical ballads, with other poems', 'The Rime of the Ancient Mariner', 'The Lucy poems', 'Biographia Literaria', 'On the constitution of the Church and State', "Coleridge's writings.", 'Poems of nature and romance, 1794-1807', 'Samuel Taylor Coleridge', 'The portable Coleridge', 'The Collected Works of Samuel Taylor Coleridge

In [141]:
print(topic_prop)

["Coleridge's works include poetry, plays, and literary criticism, and he is considered one of the most important figures in the Romantic movement.", 'Coleridge was influenced by various philosophers and writers, including Giambattista Vico, Baruch Spinoza, Friedrich Wilhelm Joseph Schelling, Emanuel Swedenborg, Immanuel Kant, John Milton, William Shakespeare, Robert Burns, Thomas Browne, and Algernon Charles Swinburne.', "Coleridge attended Christ's Hospital and the University of Cambridge, where he studied literature and philosophy.", "Coleridge's professions included poet, philosopher, and critic.", "Coleridge's peers included Thomas De Quincey, William Wordsworth, and Charles Lamb, who were also influential figures in the Romantic movement."]


In [128]:
[i[i.find(" ")+1:] for i in topic_prop.split('\n') if re.match("^[0-99]. ", i)]

['The Appalachian Mountains are a mountain range in the eastern United States, stretching from Quebec, Canada, to Alabama, USA.',
 'The Appalachian Mountains have several passes, including Grafton Notch, Cumberland Gap, and Franconia Notch.',
 'The Appalachian Mountains cover an area of approximately 190,886 square miles.',
 'The Appalachian Mountains are located in North America.',
 'The Appalachian Mountains have a geolocation of 2037.0, 40.0, -78.0.']

In [115]:
line_breaks = ['\n{}. '.format(i+1) for i in range(5)]

a = [i for i in topic_prop.split('\n')]


In [124]:
[i[i.find(" ")+1:] for i in topic_prop.split('\n') if re.match("^[0-99]. ", i)]

['The Appalachian Mountains are a mountain range in the eastern United States, stretching from Quebec, Canada, to Alabama, USA.',
 'The Appalachian Mountains have several passes, including Grafton Notch, Cumberland Gap, and Franconia Notch.',
 'The Appalachian Mountains cover an area of approximately 190,886 square miles.',
 'The Appalachian Mountains are located in North America.',
 'The Appalachian Mountains have a geolocation of 2037.0, 40.0, -78.0.']

In [134]:
sparql = """
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?r
WHERE {
?x ?r ?e .
}
"""
len(execurte_sparql(sparql))

10000